# 05: Delaunay Triangulation

*Authors: Felix Espey*

This notebook serves as supplementary learning material for the course **Geometric Algorithms**.
It showcases and explains implementations of algorithms presented in the corresponding lecture, and elaborates on some practical considerations concerning their use.
Furthermore, it offers interactive visualisations and animations.

## Table of Contents

1. Introduction
2. Concepts
   1. Empty Circle Property
   2. Edge Flips
3. Randomised Incremental Construction
4. Voronoi diagrams
5. References

## 1. Introduction

This chapter focuses mainly on the **Delaunay Triangulation**, but also contains a small section on the connected topic **Voronoi diagrams**.

We start by laying out some concepts from the lecture that are important to the Delaunay Triangulation and then go over the algorithm to construct the Delaunay Triangulation from the lecture.

## 2. Concepts

Before we can look at the algorithm, we need to go over some basic concepts needed to define what a Delaunay Triangulation is.

## 1. Empty Circle Property

The most important concept for the Delaunay Triangulation is the **Empty Circle Property**. First, lets define some basic terms:
- a circle is defined by two parts: the interior and the boundary
- a circle is empty if no points are within its interior
- a circle C<sub>p,q,r</sub> is the the circle whose boundary goes through the points p, q and r
  - since p,q and r are on the boundary, the circle is still empty

The empty Circle Property states that
1. given a Delaunay Triangulation, the circumcircle C<sub>p,q,r</sub> of all triangles pqr is empty
2. a Triangulation in which the circumcircles C<sub>p,q,r</sub> of all triangles are empty is a Delaunay Triangulation

# 2. Edge Flips

Edge flips are used to increase the size of the smallest angle between two triangles by flipping the shared edge of those triangles.
In the image below TODO, the four points *p,r,q,s* form the two triangles *pqr* and *pqs* with the shared edge *pq*.
By flipping the edge *pq* to *sr*, the initial triangles are destroyed and two new triangles *prs* and *qrs* are formed, which have a larger smallest angle. If flipping an edge leads to an increase of the minimal angle of the triangulation, the edge is called illegal. A triangulation with no illegal edges is called legal.

The edge *pq* between *pqr* and *pqs* is illegal if the circumcircle C<sub>*pqr*</sub> of *pqr* is not empty.
The image below shows an example of an edge flip increasing the minimal angle in a triangulation.


<img style='float: left;' src='./images/06-image01_placeholder.PNG'>

# 3. Randomized Incremental Construction

To construct the Delaunay Triangulation for a set of points we will use a randomized algorithm by inserting the points one by one. After each insertion the triangulation is leagilized by flipping all illegal edges. 

# 4. Voronoi Diagramms